# 한국어 word2vec 만들기

In [ ]:
# Install konlpy
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

네이버 뉴스 데이터 가져오기

In [1]:
from google.colab import drive
drive.mount('/content/drive') # 원하는 세부 경로로 변경이 불가능합니다.

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

csv_list = ['/content/drive/MyDrive/data/result_231210_0111.csv',
            '/content/drive/MyDrive/data/result_231210_0206.csv',
            '/content/drive/MyDrive/data/result_231210_0239.csv',
            '/content/drive/MyDrive/data/result_231210_0127.csv',
            '/content/drive/MyDrive/data/result_231210_0221.csv']
news_df = pd.DataFrame()
for csv in csv_list:
    df_data = pd.read_csv(csv)
    news_df = news_df.append(df_data)

print('전체 리뷰 수:', len(news_df))
news_df = news_df.dropna()
eco_news_df = news_df[news_df['category'] == '경제']
eco_news = eco_news_df['document'].tolist()
# eco_news[:10]

전체 리뷰 수: 2316


<ipython-input-3-159200164663>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append(df_data)
<ipython-input-3-159200164663>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append(df_data)
<ipython-input-3-159200164663>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append(df_data)
<ipython-input-3-159200164663>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append(df_data)
<ipython-input-3-159200164663>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news

In [15]:
len(eco_news)

713

In [6]:
import pickle

# save
with open('eco_news.pickle', 'wb') as f:
    pickle.dump(eco_news, f, pickle.HIGHEST_PROTOCOL)
# load
# with open('data.pickle', 'rb') as f:
    # eco_news = pickle.load(f)

Text preprocessing

In [7]:
import re

def tokenize_and_stemming(sentence, tokenizer):
    """
    토크나이저로 복수의 문장을 한 개의 리스트로 만든다.
    :args
        sentence: 한국어 문단
        tokenizer: konlpy.tag 라이브러리의 토크나이저
    :return
        sentence_tokenized: 한국어 토큰 e.g. [통계청, 인구, 동향, 작년, 개월, 인구, 자연, 감소, 세종, 연합뉴스, 이준...
    """
    sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence)

    raw_pos_tagged = tokenizer.pos(sentence, stem=True) # POS Tagging with stemming

    sentence_tokenized = []
    for token, pos in raw_pos_tagged: # ex) word <- ('서울', 'Noun')
        # pos 기준 제거 대상 : ["Josa", "Eomi", "Punctuation", "Foreign", "Number", "Suffix", "Determiner"]
        # 리뷰 텍스트 상 형태소의 순서를 고려하는 경우 의미를 갖게 됨 : "VerbPrefix" (잘/안/못), "Conjunction"(하지만/그러나/그리고), "KoreanParticle"(ㅋㅋㅋ/ㅎㅎㅎ/ㅠㅠ)
        if (len(token) != 1) & (pos in ["Noun", "VerbPrefix", "Verb", "Adverb", "Adjective", "Conjunction", "KoreanParticle"]):
            sentence_tokenized.append(token)

    return sentence_tokenized

In [13]:
from konlpy.tag import Okt
import tqdm

okt = Okt()
tokenized_sentences = []

for sentences in tqdm.tqdm(eco_news):
    sentence_list = sentences.split('.') # 문장 단위로 분리
    for sentence in sentence_list:
        try:
            tokenized_sentences.append(tokenize_and_stemming(sentence, okt))
        except:
            print("Error occured at :", sentence)
            tokenized_sentences.append([])

100%|██████████| 713/713 [01:18<00:00,  9.09it/s]


In [46]:
tokenized_sentences[:10]

[['통계청', '인구', '동향', '광역시', '일제', '하락', '광주'],
 ['줄다', '하락', '최대', '인구', '개월', '자연', '감소', '혼인', '건수', '넘다', '줄다', '출산율'],
 ['명대', '떨어지다', '수도', '서울', '지하철', '호선', '임산부', '배려', '비어', '있다'],
 ['통계청', '발표', '인구', '동향', '따르다', '오다', '합계', '출산율', '전보'],
 ['낮아지다'],
 ['기록', '하다'],
 ['임대', '기자', '올해', '합계', '출산율'],
 ['기록', '하다'],
 ['여성', '하다', '평생', '낳을', '예상', '되다', '출생', '아가'],
 ['그치다']]

In [51]:
# save
result_file = 'preprocessed_word2vec.pickle'
with open(result_file, 'wb') as f:
    pickle.dump(tokenized_sentences, f, pickle.HIGHEST_PROTOCOL)
print(result_file + ' 파일 저장됨!')

preprocessed_word2vec.pickle 파일 저장됨!


## CBOW word2vec 모델 학습하기
gensim 패키지로 word2vec을 학습시킨다.
- CBOW: 문제의 DBOW는 doc2vec에 해당되는 옵션이다.
- Skip-gram

In [38]:
from gensim.models import Word2Vec
# sg=0 -> CBOW
model_cbow = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=5, workers=4, sg=0)
model_cbow.wv.vectors.shape

(4309, 100)

In [ ]:
model_filename = 'cbow_word2vec.model'
model_cbow.save(model_filename)
print(model_filename + ' saved!')

cbow_word2vec.model saved!


Test 유사어 찾기

In [40]:
print(model_cbow.wv.most_similar("주가"))

[('달러', 0.9365742206573486), ('작년', 0.9342687129974365), ('평균', 0.924926221370697), ('매출', 0.9232419729232788), ('정점', 0.922761082649231), ('오른', 0.9204503893852234), ('포인트', 0.9200392961502075), ('시가총액', 0.9184390902519226), ('벌어지다', 0.9183456897735596), ('급등', 0.9183215498924255)]


## Skip-gram word2vec 모델 학습하기

In [41]:
# sg=1 -> skip-gram
model_sg = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=5, workers=4, sg=1)
model_sg.wv.vectors.shape

(4309, 100)

In [42]:
model_filename = 'skip-gram_word2vec.model'
model_sg.save(model_filename)
print(model_filename + ' saved!')

skip-gram_word2vec.model saved!


Test 유사어 찾기

In [43]:
print(model_sg.wv.most_similar("주가"))

[('시가총액', 0.8364740610122681), ('달러', 0.8070789575576782), ('하루', 0.7969028353691101), ('찍다', 0.795524537563324), ('오른', 0.7944868803024292), ('코스닥', 0.7841623425483704), ('이노텍', 0.7809240221977234), ('벌어지다', 0.780060887336731), ('무역수지', 0.7799789905548096), ('국고', 0.7668311595916748)]
